In [7]:
%pip install -e ..

Obtaining file:///Users/sethvanderbijl/Coding%20Projects/DM_project_2
  Attempting uninstall: DM-project-2
    Found existing installation: DM-project-2 1.0
    Uninstalling DM-project-2-1.0:
      Successfully uninstalled DM-project-2-1.0
  Running setup.py develop for DM-project-2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from json import load
from data.load import load_data
import os
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, StackingClassifier, StackingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, HuberRegressor, LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import std
import time
import matplotlib.pyplot as plt
import copy
from helpers.ranking import to_ranking


/usr/local/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:

# Load data
df = load_data(add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)

No cache for this specific request, start loading base df from disk.


  0%|          | 0/6 [00:00<?, ?it/s]

generating dayparts
1 / 6 Generating datetimes for early_morning


 17%|█▋        | 1/6 [00:21<01:45, 21.10s/it]

2 / 6 Generating datetimes for morning


...

100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


Feature engineering columns with operations


  0%|          | 0/500 [00:00<?, ?it/s]

saving result of specific arguments to cache


100%|██████████| 500/500 [00:32<00:00, 15.32it/s]


In [3]:
# Split into target and predictors
y = df['booking_bool']
X = df.drop(['booking_bool','click_bool', 'position', 'gross_bookings_usd', 'date_time'], axis=1)
X = X.fillna(X.mean())
del df

In [5]:
# Set up stakcing regressor
# rf = RandomForestRegressor()
# rf= StackingRegressor([('rf',RandomForestRegressor()), ('br',BayesianRidge()), ('ab',AdaBoostRegressor())], verbose=3, n_jobs=7)
rf=RandomForestRegressor()
rf.fit(X, y)

In [ ]:
del y

# Split 
test = load_data(mode='our_test', add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)

# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_test = test.drop('date_time', axis=1)
X_test = X_test.fillna(X.mean()) #Mean of x or mean of x_test?
del X

In [ ]:
# Evaluate
preds = rf.predict(X_test)

# [print(a,b, c) for a,b, c in zip(test['srch_id'], test['prop_id'],preds)]

ranking = to_ranking(X_test, preds)
score = gcbs(ranking, test)
ranking.to_csv('test_ranking.csv', index=False)

# mse = mean_squared_error(y_test, preds)
# mae = mean_absolute_error(y_test, preds)
# print('\nerrors\n\n', mse, mae)